### 4. Fuzzy Logic Toolbox implementeren

    ~Peter Heemskerk
    
Hieronder de standaard uit het LAB vrekregen code voor een fuzzy logic implementatie. De classes zijn zo gedefinieerd dat je in een aantal stappen een eenvoudige implementatie van een Fuzzy Logic systeem kunt definieren. 

stap a. creeer membership functie mbv. class TriangularMF of class TrapezoidalMF. 
        voorbeeld: 
            triangular_mf = TriangularMF("medium", 150, 250, 350) 
            
stap b. creer input en output functes met class Input(Variable) of class Output(Variable). 
        voorbeeld: 
            mfs_income = [TrapezoidalMF("low", -100, 0, 200, 400), TriangularMF("medium", 200, 500, 800)]
            income = Input("income", (0, 1000), mfs_income)\
            
stap c. zet deze inputfuncties en outputfuncies in de lijst met inputs en outputs.
        voorbeeld: 
            inputs = [income, quality]
            output = money
            
stap d. definieer regels middels class Rule:
        voorbeeld: 
            rule1 = Rule(1, ["low", "amazing"], "and", "low")

stap e. zet alle regels in een class Rulebase: 
        voorbeeld: 
            rules = [rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9]
            rulebase = Rulebase(rules).

stap f. je kunt nu firing strenght van deze regel oproepen: 
            rule1.calculate_firing_strength([200, 6.5], inputs), 
        en de totale firing strenghts (!):  
            rulebase.calculate_firing_strengths(datapoint, inputs)
            
### 4.0 Wijzgigingen
Er zijn twee wijzigingen doorgevoerd t.o.v. de standaard LAB versie: 
(a). werken met onvolledige antecedenten. Door bij regel "" op te nemen, wordt deze anthecedent niet gebruikt. 
(b). het is nu mogelijk met 2 of meer output variabelen te werken. 
Nog doen (Peter): 
(c). testen of het ook goed blijft werken als in regel maar 1 output variabele in de consequent wordt meegenomen. 
(d). defuzzification: invoeren van de centroid of mean of max defuzzification methode

### 4.0 Wijze van aanroepen. 
Zie laatste cel: met een datapoint=[lijst met 7 getallen tussen 0 en 100] kun je een object A in de class Reasoner definieren en met A.inference komen dan de outputs te voorschijn. 


#### 4.1 Membership Functions

Membership functions are used to fuzzify crisp inputs, representing an item's membership to a class, with 0 meaning no membership and 1 meaning the item is a perfect prototype of a class.

During the Fuzzy Logic crash course we have seen 4 basic types of membership functions: triangular, trapezoidal, gaussian and generalized bell shaped membership functions. Those membership functions are represented in the image:

In [4]:
import math
import numpy as np
from collections import defaultdict, Counter

class TriangularMF:
    """Triangular fuzzy logic membership function class."""
    def __init__(self, name, start, top, end):
        self.name = name
        self.start = start
        self.top = top
        self.end = end

    def calculate_membership(self, x):
        if x <= self.start: 
            y = 0
        if x > self.start and x <= self.top: 
            y = (x-self.start)/(self.top-self.start)
        if x > self.top and x <= self.end:
            y = (self.end - x)/(self.end - self.top)
        if x > self.end:
            y = 0
        return y
        
class TrapezoidalMF:
    """Trapezoidal fuzzy logic membership function class."""
    def __init__(self, name, start, left_top, right_top, end):
        self.name = name
        self.start = start
        self.left_top = left_top
        self.right_top = right_top
        self.end = end

    def calculate_membership(self, x):
        if x <= self.start: 
            y = 0
        if x > self.start and x <= self.left_top: 
            y = (x - self.start)/(self.left_top - self.start)
        if x > self.left_top and x <= self.right_top:
            y = 1
        if x > self.right_top and x <= self.end:
            y = (self.end - x)/(self.end - self.right_top)
        if x > self.end:
            y = 0
        return y


In [5]:
# Test your implementation by running the following statements
# Enter your answers in the Google form to check them, round to two decimals

triangular_mf = TriangularMF("medium", 150, 250, 350)
print(triangular_mf.calculate_membership(100))
print(triangular_mf.calculate_membership(249))
print(triangular_mf.calculate_membership(300))

trapezoidal_mf = TrapezoidalMF("bad", -10, 0, 20, 40)
print(trapezoidal_mf.calculate_membership(0))
print(trapezoidal_mf.calculate_membership(10))
print(trapezoidal_mf.calculate_membership(20))

0
0.99
0.5
1.0
1
1


### 4.2. Inputs and output
Theory

The inputs and output of a FLS are represented through linguistic variables, which are variables whose values are words rather than numbers. A value of a linguistic variable is called a linguistic term.

An example of a linguistic variable is 'income' (a variable we're using in the system we're programming), with the linguistic terms 'low', 'medium', 'high'.
Practice

Now we are going to define input and output variables, which are a collection of multiple membership functions.

A variable's membership functions (self.mfs) should be a list of membership functions. Define the input variables income and quality and the output variable money with the name, range and membership functions represented in the image.

In [6]:
class Variable:
    """General class for variables in an FLS."""
    def __init__(self, name, range, mfs):
        self.name = name
        self.range = range
        self.mfs = mfs

    def calculate_memberships(self, x):
        """Test function to check whether
        you put together the right mfs in your variables."""
        return {
            mf.name : mf.calculate_membership(x)
            for mf in self.mfs
        }

    def get_mf_by_name(self, name):
        for mf in self.mfs:
            if mf.name == name:
                return mf

class Input(Variable):
    """Class for input variables, inherits 
    variables and functions from superclass Variable."""
    def __init__(self, name, range, mfs):
        super().__init__(name, range, mfs)
        self.type = "input"

class Output(Variable):
    """Class for output variables, inherits 
    variables and functions from superclass Variable."""
    def __init__(self, name, range, mfs):
        super().__init__(name, range, mfs)
        self.type = "output"

In [7]:
# Input variable: personal
mfs_personal = [TrapezoidalMF("possible", -10, 0, 20, 40), TriangularMF("probable", 20, 50, 80), TrapezoidalMF("certain", 60, 80, 100, 120)]
personal = Input("personal", (0, 100), mfs_personal)

# Input variable: space
mfs_space = [TrapezoidalMF("possible", -10, 0, 20, 40), TriangularMF("probable", 20, 50, 80), TrapezoidalMF("certain", 60, 80, 100, 120)]
space = Input("space", (0, 100), mfs_space)

# Input variable: financial
mfs_financial = [TrapezoidalMF("possible", -10, 0, 20, 40), TriangularMF("probable", 20, 50, 80), TrapezoidalMF("certain", 60, 80, 100, 120)]
financial = Input("financial", (0, 100), mfs_financial)

# Input variable: traffic
mfs_traffic = [TrapezoidalMF("possible", -10, 0, 20, 40), TriangularMF("probable", 20, 50, 80), TrapezoidalMF("certain", 60, 80, 100, 120)]
traffic = Input("traffic", (0, 100), mfs_traffic)

# Input variable: tax
mfs_tax = [TrapezoidalMF("possible", -10, 0, 20, 40), TriangularMF("probable", 20, 50, 80), TrapezoidalMF("certain", 60, 80, 100, 120)]
tax = Input("tax", (0, 100), mfs_tax)

# Input variable: agitation
mfs_agitation = [TriangularMF("neutral", 0, 20, 40), TriangularMF("dissatisfaction", 20, 50, 80), TriangularMF("anger", 20, 50, 80), TrapezoidalMF("turmoil", 60, 80, 100, 120)]
agitation = Input("agitation", (0, 100), mfs_agitation)

# Input variable: action
mfs_action =[TrapezoidalMF("suggest", -10, 0, 20, 40), TriangularMF("needed", 20, 50, 80), TrapezoidalMF("now", 60, 80, 100, 120)]
action = Input("action", (0, 100), mfs_action)

# Output variable: department
mfs_department = [TriangularMF("informatie", 0, 100, 200), TriangularMF("belastingen", 200, 300, 400), TriangularMF("parkeren", 400, 500, 600), TriangularMF("werkeninkomen", 600, 700, 800), TriangularMF("generalaffairs", 800, 900, 1000)]
department = Output("department", (0, 1000), mfs_department)

# Output variable: priority
mfs_priority = [TriangularMF("execution", 0, 100, 200), TriangularMF("management", 200, 300, 400), TriangularMF("political", 400, 500, 600)]
priority = Output("priority", (0, 1000), mfs_priority)

inputs = [personal, space, financial, traffic, tax, agitation, action]
output = [department, priority]

In [8]:
# Test your implementation by running the following statements
# Enter your answers in the Google form to check them, round to two decimals

# print(personal.calculate_memberships(69))
print(agitation.calculate_memberships(-10))
# print(department.calculate_memberships(22))

# print(priority.name, priority.range)
# print('inputs:', inputs[0].name, inputs[0].calculate_memberships(489))

{'turmoil': 0, 'dissatisfaction': 0, 'neutral': 0, 'anger': 0}


## 4.3 Fuzzy Rules

### Theory

A fuzzy IF-THEN rule is composed of an antecedent and a consequent, like the implications you have seen with propositional and first order logic.
- In the antecedent conditions for input variables are connected with an operator (AND, OR, NOT), e.g. "IF x1 is mf1 AND x2 is mf3", or "IF x1 is mf1 OR x2 is mf2".
- The AND operator represents taking the intersection of fuzzy sets, which can be accomplished by choosing a T-Norm operation, such as minimum.
- The OR operator represents taking the union of fuzzy sets, which can be accomplished by choosing a T-Conorm operation, such as maximum.
- The NOT operator, the complement, is calculated by 1 minus a membership value. For example: NOT x1 is mf1, would be 1 - (membership of x1 to mf1).
- By the use of the AND, NOT and OR we combine the different parts of the antecedent into a single number, which is the firing strength of the antecedent.
- The consequent represents an action that we undertake if the rule fires.

### Practice

We are going to add some simple rules for our FLS: complete rules that do not have mixed operators. Here we represent a rule through 3 variables:
- <b>Antecedent</b>, represented as a list of names of membership functions. The index of the name corresponds to the variable it belongs to, for example: ["medium", "low"], where "medium" belongs to the first variable in *inputs* and "low" corresponds to the second variable in *inputs*.
- <b>Operator</b>: "and" or "or", let's choose "and".
- <b>Consequent</b>: a string corresponding one of the membership functions of your output variable, for example "high".

These three variables would then compose the rule "IF income is medium AND quality is low THEN money is high."

Complete the *calculate_firing_strength()*, that should function and check your answers by running the test statements.

In [34]:
class Rule:
    """Fuzzy rule class, initialized with an antecedent (list of strings),
    operator (string) and consequent (string)."""
    def __init__(self, n, antecedent, operator, consequent):
        self.number = n
        self.antecedent = antecedent
        self.operator = operator
        self.consequent = consequent
        self.firing_strength = 0

    def check_antecedents(self):
        # nog eens doen
        return
    
    def calculate_firing_strength(self, datapoint, inputs):
        # choosen min operator for T-norm
        i = 0
        self.firing_strength = 9999999999999999999
        for input_ms in self.antecedent:
            # print (i, input_ms, datapoint[i])
            if input_ms == "":
                msvalue = 1     # bij missing input in rule, telt deze mee voor 1
            else:
                mslijst = inputs[i].calculate_memberships(datapoint[i])
                msvalue = mslijst[input_ms]
            self.firing_strength = min(self.firing_strength, msvalue)
            # print (msvalue, self.firing_strength)
            i += 1
        return self.firing_strength

In [35]:
# Test your implementation by checking the following statements
# Enter your answers in the Google form to check them, round to two decimals

rule1 = Rule(1, ["probable", "", "probable", "", "", "neutral", "needed"], "and", ["belastingen", "execution"])
print(rule1.calculate_firing_strength([50, 10, 60, 30, 50, 0, 50], inputs))
print(rule1.calculate_firing_strength([0, 10, 60, 30, 20, 0, 50], inputs))

rule2 = Rule(2, ["", "certain", "", "", "", "", ""], "and", ["parkeren", "management"])
print(rule2.calculate_firing_strength([100, 70, 70, 70, 60, 40, 70], inputs))
print(rule2.calculate_firing_strength([70, 75, 30, 20, 50, 10, 60], inputs))

0
0
0.5
0.75


## 4.4 Fuzzy Rulebase


### Theory

A rulebase is simply a collection of all rules of the system!

### Practice

Our fuzzy rulebase is a collection of all rules. Create the following rules and initalize the fuzzy rulebase:
- IF income is low AND quality is amazing THEN money is low
- IF income is medium AND quality is amazing THEN money is low
- IF income is high AND quality is amazing THEN money is low
- IF income is low AND quality is okay THEN money is low
- IF income is medium AND quality is okay THEN money is medium
- IF income is high AND quality is okay THEN money is medium
- IF income is low AND quality is bad THEN money is low
- IF income is medium AND quality is bad THEN money is medium
- IF income is high AND quality is bad THEN money is high

Implement the *calculate_firing_strengths()* function that collects the highest firing strength found per membership function of the output variable in a dictionary or Counter object.
For example, if the firing strengths for the rules listed above are 0, 0, 0, 0.5, 0.25, 0, 0, 0, 0 the result would look like this: *{"low":0.5, "medium":0.25, "high"0}*.

Check the correctness of your function with the testing statements.

In [36]:
from collections import Counter

class Rulebase:
    """The fuzzy rulebase collects all rules for the FLS, can
    calculate the firing strengths of its rules."""
    def __init__(self, rules):
        self.rules = rules

    def calculate_firing_strengths(self, datapoint, inputs, outputindex):
        result = Counter()
        for i, rule in enumerate(self.rules):
            consequent = rule.consequent[outputindex]
            # check of consequent bestaat !!
            if consequent != "":
                fs = rule.calculate_firing_strength(datapoint, inputs)
                if fs > result[consequent]:
                    result[consequent] = fs
            # print('RULE', i+1, consequent, result[consequent])
        return result

In [37]:
# Add the rules listed in the question description
# Your code here
rule1 = Rule(1, ["", "", "", "", "certain", "", ""], "and", ["belastingen", "execution"])
rule2 = Rule(2, ["certain", "", "certain", "", "", "", ""], "and", ["werkeninkomen", "execution"])
rule3 = Rule(3, ["certain", "", "", "certain", "", "", ""], "and", ["parkeren", "execution"])
rule4 = Rule(4, ["", "certain", "", "", "", "", ""], "and", ["generalaffairs", "execution"])
rule5 = Rule(5, ["", "", "", "", "", "neutral", ""], "and", ["informatie", "execution"])
rule6 = Rule(6, ["", "", "", "", "", "", "needed"], "and", ["informatie", "execution"])
rule7 = Rule(7, ["", "", "", "", "", "", "now"], "and", ["generalaffairs", "management"])
rule8 = Rule(8, ["", "", "", "", "", "turmoil", ""], "and", ["generalafffairs", "political"])
rule9 = Rule(9, ["", "", "", "", "", "anger"], "and", ["generalaffairs", "management"])

# print('testfs: ', rule1.calculate_firing_strength([234, 7.5], inputs))

rules = [rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9]

rulebase = Rulebase(rules)

In [38]:
# Test your implementation of calculate_firing_strengths()
# Enter your answers in the Google form to check them, round to two decimals

datapoint = [80, 3, 4, 30, 80, 30, 60]
print(rulebase.calculate_firing_strengths(datapoint, inputs, outputindex=0))

datapoint = [34, 75, 33, 44, 55, 66, 77]
print(rulebase.calculate_firing_strengths(datapoint, inputs, outputindex=1))

Counter({'belastingen': 1, 'informatie': 0.6666666666666666, 'generalaffairs': 0.3333333333333333})
Counter({'management': 0.85, 'execution': 0.75, 'political': 0.3})


## 4.5. Inference (aggregation and defuzzification)

### Theory

In the Fuzzy Inference all parts of the fuzzy system come together: we are mapping an input to an output in the following way:
1. Fuzzify the input.
2. Calculate the firing strengths for the rules.
3. Use the firing strength to determine the contribution of the consequent.
4. Aggregate / collect all consequents.
5. Defuzzify

As already mentioned, there is Mamdani type inference and Takagi-Sugeno-Kang (TSK) type inference:
- With Mamdani type inference we represent the consequents of fuzzy rules as fuzzy sets (using membership functions). We use a rule's firing strength to adapt the height of the membership function in the consequent, using the implication operator (minimum or product). The consequents are then aggregated into one area (taking the maximum of all consequents for the entire input range), on which we apply a defuzzification method, such as 'largest of max', 'smallest of max' or 'centroid'.
- With TSK type inference we represent the consequents as a function of the input variables, or a constant. To combine the consequents into one output number we calculate a weighted average, where the weights are the rules' firing strengths.

<img src="https://i.imgur.com/q5lzbsZ.png"></img>
<img src="https://i.imgur.com/Yl20dJL.png"></img>

In the following image multiple defuzzification methods are visualized:
<img src="http://access.feld.cvut.cz/storage/201208252026_obr-15.png"></img>

### Practice

We will finalize our system using Mamdani type inference by performing the following three steps:
1. Gathering the largest firing strength per membership function of the output variable (implemented in your rulebase in Step 3)
2. Discretizing the range of your output variable and applying the aggregation method (<b>max</b>): for every bin you find the maximum fuzzy membership value. Notice that the membership functions of the output variable are `cut off' according to the firing strengths, with the implication method (<b>min</b>).
To accomplish this we perform two steps:
    - First we find where the aggregated area starts and ends on the x-axis
    - Second we discretize the area between start and end into 201 points (thus representing the area in 200 bins)
3. Applying two defuzzification methods: implement smallest of max (<b>som</b>) and largest of max (<b>lom</b>).

In [39]:
class Reasoner:
    def __init__(self, rulebase, inputs, output, outputindex, n_points, defuzzification):
        self.rulebase = rulebase
        self.inputs = inputs
        self.output = output
        self.outputindex = outputindex
        self.discretize = n_points
        self.defuzzification = defuzzification

    def inference(self, datapoint):
        # 1. Calculate the highest firing strength found in the rules per 
        # membership function of the output variable
        # looks like: {"low":0.5, "medium":0.25, "high":0}
        firing_strengths = rulebase.calculate_firing_strengths(datapoint, inputs, outputindex)

        # 1a. Checks that all consequent names (as stored in firingstrenghts) exist in variable definition
        self.check_consequents(firing_strengths, outputindex)
        
        # 2. Aggragate and discretize
        # looks like: [(0.0, 1), (1.2437810945273631, 1), (2.4875621890547261, 1), (3.7313432835820892, 1), ...]
        input_value_pairs = self.aggregate(firing_strengths)

        # 3. Defuzzify
        # looks like a scalar
        crisp_output = self.defuzzify(input_value_pairs)
        return crisp_output

    def aggregate(self, firing_strengths):  
        # First find where the aggrageted area starts and ends
        agg_start = self.output[outputindex].range[0]
        agg_end = self.output[outputindex].range[1]
        
        # Second discretize this area and aggragate
        # note: if a typo is made in consequent-names, these values will be disregarded without notice !!
        aantal = self.discretize
        breedte = (agg_end - agg_start)/(aantal-1)
        input_value_pairs = []
        for n in range(aantal):
            x = agg_start + n * breedte
            # mslijst bevat de membership waarde van de output variabele (met gegeven outputindex)
            mslijst = self.output[outputindex].calculate_memberships(x)
            value = 0
            for ms in mslijst: 
                ms_min = min(mslijst[ms], firing_strengths[ms])
                value = max(ms_min, value)
            input_value_pairs.append((x, value))
        return input_value_pairs

    def check_consequents(self, firing_strengths, outputindex):
        agg_start = self.output[outputindex].range[0]                # arbitrary point in domain
        mslijst = self.output[outputindex].calculate_memberships(agg_start)
        for ms in firing_strengths:
            if ms not in mslijst:
                print('WARNING - consequent:', ms, 'does not match outputdefinition')
        return
    
    def defuzzify(self, input_value_pairs):
        maxms = 0
        crisp_value = 9999
        # crisp_value = 9999 is eigenlijk foutsituatie
        if self.defuzzification =="som":    
            for value_pair in input_value_pairs:
                if value_pair[1]>maxms:
                    maxms = value_pair[1]
                    crisp_value = value_pair[0]
        elif self.defuzzification == "lom":
            for value_pair in input_value_pairs:
                if value_pair[1]>=maxms:
                    maxms = value_pair[1]
                    crisp_value = value_pair[0]
        elif self.defuzzification == 'centroid':
            teller = 0
            noemer = 0
            for value_pair in input_value_pairs:
                teller += value_pair[0]*value_pair[1]
                noemer += value_pair[1]
            if noemer == 0:
                # mag beter
                crisp_value = 0
            else:
                crisp_value = teller/noemer
        return crisp_value

In [40]:
for outputindex in range(2):
    thinker = Reasoner(rulebase, inputs, output, 0 , 201, "centroid")
    datapoint = [80, 3, 4, 30, 80, 30, 60]
    print(output[outputindex].name, ':', round(thinker.inference(datapoint)))

for outputindex in range(2):    
    thinker = Reasoner(rulebase, inputs, output, outputindex, 101, "centroid")
    datapoint = [80, 100, 33, 0, 20, 44, 10]
    print(output[outputindex].name, ':', round(thinker.inference(datapoint)))

for outputindex in range(2):
    thinker = Reasoner(rulebase, inputs, output, outputindex, 201, "som")
    datapoint = [10, 22, 40, 50, 60, 30, 0]
    print(output[outputindex].name, ':', round(thinker.inference(datapoint)))

for outputindex in range(2):
    thinker = Reasoner(rulebase, inputs, output, outputindex, 201, "centroid")
    datapoint = [0, 0, 0, 0, 0, 0, 0]
    print(output[outputindex].name, ':', round(thinker.inference(datapoint)))

for outputindex in range(2):
    thinker = Reasoner(rulebase, inputs, output, outputindex, 101, "som")
    datapoint = [100, 100, 100, 100, 100, 100, 100]
    print(output[outputindex].name, ':', round(thinker.inference(datapoint)))
    
for outputindex in range(2):
    thinker = Reasoner(rulebase, inputs, output, outputindex, 201, "lom")
    datapoint = [55, 33, 44, 22, 11, 99, 0]
    print(output[outputindex].name, ':', round(thinker.inference(datapoint)))

department : 364
priority : 171
department : 900
priority : 198
department : 50
priority : 50
department : 0
priority : 0
WARNING - consequent: generalafffairs does not match outputdefinition
department : 300
priority : 100
WARNING - consequent: generalafffairs does not match outputdefinition
department : 1000
priority : 500
